# Caso de estudio 2: optimización de la composición de una barra nutricional

- Queremos formular una barra nutricional con una mezcla específica de ingredientes para cumplir con ciertos requisitos nutricionales, mientras se mantiene dentro de un presupuesto.
- Cada ingrediente tiene un costo por unidad y contribuye a la barra con una cantidad específica de nutrientes.

## Datos de ingredientes

| Ingrediente           | Costo por gramo | Proteínas (g/100g) | Carbohidratos (g/100g) | Grasas (g/100g) | Calorías (kcal/100g) |
|---------------------------|---------------------|------------------------|----------------------------|---------------------|--------------------------|
| **Proteína de Suero**         | \$0.10               | 80                     | 5                          | 2                   | 350                      |
| **Avena**                 | \$0.02               | 15                     | 60                         | 6                   | 370                      |
| **Mantequilla de Almendra**| \$0.15               | 25                     | 20                         | 50                  | 600                      |
| **Azúcar**                | $0.01               | 0                      | 100                        | 0                   | 400                      |

## Rangos

| **Ingrediente**           | **Rango de Cantidad (gramos)** |
|---------------------------|-------------------------------|
| **Proteína de Suero**     | 10 - 50                       |
| **Avena**                 | 20 - 100                      |
| **Mantequilla de Almendra**| 5 - 30                       |
| **Azúcar**                | 0 - 20                        |



## Variables

- $v_1$: Cantidad de proteína de suero en la barra (en gramos)
- $v_2$: Cantidad de avena en la barra (en gramos)
- $v_3$: Cantidad de mantequilla de almendra en la barra (en gramos)
- $v_4$: Cantidad de azúcar en la barra (en gramos)

## Restricciones

  - **Proteínas**: Al menos 20 gramos. Proteína >=20.
  - **Carbohidratos**: Al menos 30 gramos. Carbohidratos >= 30.
  - **Grasas**: Al menos 10 gramos. Grasas >=10.02.
  - **Calorías**: Entre 250 y 350 kcal. 250 <= Calorías >= 350.


## Función Objetivo

- Minimizar el costo total de la barra:

  $$
  \text{Costo Total} = 0.10v_1 + 0.02v_2 + 0.15v_3 + 0.01v_4
  $$

Donde:
- $v_1$: Cantidad de proteína de suero en la barra (en gramos).
- $v_2$: Cantidad de avena en la barra (en gramos).
- $v_3$: Cantidad de mantequilla de almendra en la barra (en gramos).
- $v_4$: Cantidad de azúcar en la barra (en gramos).

## Función de Aptitud

$$
   \text{Aptitud} = \frac{1}{\text{Costo Total} + \text{Penalización}}
$$

Donde la penalización es una suma de las penalizaciones aplicadas por incumplir los requisitos nutricionales y el rango de calorías.

- **Penalización por no cumplir los requerimientos**: Si la mezcla no cumple con alguno de los requisitos nutricionales, se aplica una penalización al costo total:
     - Penalización por cada gramo de proteína faltante.
     - Penalización por cada gramo de carbohidrato o grasa excedente o faltante.
     - Penalización por cada caloría fuera del rango deseado.




In [1]:
#!pip install pandas numpy gdown

### 1. Importación de bibliotecas y lectura de datos

- Primero, importamos las bibliotecas necesarias.
- Leemos los datos desde un archivo Excel que contiene información sobre los ingredientes y los rangos de valores permitidos para cada ingrediente.
- Extraemos la información sobre los ingredientes, costos, proteínas, carbohidratos, grasas y calorías.


In [2]:
#import gdown
import pandas as pd
import numpy as np

# URL del archivo en Google Drive
# file_id = '15QuX1fliafQbrJApky6Z0SUIli0dxXo6'
# url = f'https://drive.google.com/uc?id={file_id}'
# filename = 'barra_nutricional.xlsx'

# Archivo
filename = 'barra_nutricional.xlsx'

# Descargar el archivo
# gdown.download(url, filename, quiet=False)

# Leer datos desde el archivo descargado
ingredientes_df = pd.read_excel(filename, sheet_name='ingredientes')
rangos_df = pd.read_excel(filename, sheet_name='rangos')

# Datos de ingredientes
ingredientes = ingredientes_df['Ingrediente'].values
precios = ingredientes_df['Costo'].values
proteinas = ingredientes_df['Proteínas (g/100g)'].values
carbohidratos = ingredientes_df['Carbohidratos (g/100g)'].values
grasas = ingredientes_df['Grasas (g/100g)'].values
calorias = ingredientes_df['Calorías (kcal/100g)'].values

# Rangos
rango_minimos = rangos_df['Min'].values
rango_maximos = rangos_df['Max'].values

In [3]:
ingredientes_df.head()

,Ingrediente,Costo,Proteínas (g/100g),Carbohidratos (g/100g),Grasas (g/100g),Calorías (kcal/100g)
0,Proteína de Suero,0.10,80,5,2,350
1,Avena,0.02,15,60,6,370
2,Mantequilla de Almendra,0.15,25,20,50,600
3,Azúcar,0.01,0,100,0,400


In [4]:
rangos_df.head()

,Ingrediente,Min,Max
0,Proteína de Suero,10,50
1,Avena,20,100
2,Mantequilla de Almendra,5,30
3,Azúcar,0,20


### 2. Definición de parámetros del algoritmo genético

- Establecemos los parámetros básicos para el algoritmo genético, como el tamaño de la población, el número de generaciones, y las probabilidades de cruce y mutación.


In [5]:

# Parámetros del algoritmo genético
n_ingredientes = len(ingredientes)
poblacion_size = 20
generaciones = 100
mutacion_prob = 0.2
cruce_prob = 0.7

### 3. Función de aptitud: Cálculo de la información nutricional y costo de producción

- En esta parte necesitamos una función que calcule la cantidad total de proteínas, carbohidratos, grasas y calorías basadas en los valores del cromosoma.

- El costo total según el gramaje de cada ingrediente.

- Penalizaciones si el cromosoma no cumple con ciertos requisitos nutricionales y de calorías.

In [6]:
# Información nutricional
def calcular_nutricion(cromosoma):
    cromosoma_cada_100 = cromosoma / 100
    proteina_total = np.sum(cromosoma_cada_100 * proteinas)
    carbohidrato_total = np.sum(cromosoma_cada_100 * carbohidratos)
    grasa_total = np.sum(cromosoma_cada_100 * grasas)
    calorias_total = np.sum(cromosoma_cada_100 * calorias)
    return proteina_total, carbohidrato_total, grasa_total, calorias_total

# Función de aptitud
def calcular_aptitud(cromosoma):
    proteina_total, carbohidrato_total, grasa_total, calorias_total = calcular_nutricion(cromosoma)

    # Penalización
    penalizacion_proteinas = max(0, 20 - proteina_total) if proteina_total < 20 else max(0, proteina_total)
    penalizacion_carbohidratos = max(0, 30 - carbohidrato_total) if carbohidrato_total < 30 else max(0, carbohidrato_total)
    penalizacion_grasas = max(0, 10 - grasa_total) if grasa_total < 10 else max(0, proteina_total)
    penalizacion_calorias = max(0, 250 - calorias_total) if calorias_total < 250 else max(0, calorias_total - 350)

    penalizacion_total = penalizacion_proteinas + penalizacion_carbohidratos + penalizacion_grasas + penalizacion_calorias
    costo_total = np.sum(cromosoma * precios)
    #print(costo_total, penalizacion_total)
    aptitud = 1/ (costo_total + penalizacion_total)

    return aptitud


### 5. Inicialización de la población

- La función `inicializar_poblacion` crea una población inicial de soluciones (cromosomas) generando valores aleatorios dentro de los rangos permitidos para cada ingrediente.


In [7]:
# Inicializar población
def inicializar_poblacion(rangos_min, rangos_max, poblacion_size, n_ingredientes):
    return np.random.uniform(rangos_min, rangos_max, (poblacion_size, n_ingredientes))

poblacion_prueba = inicializar_poblacion(rango_minimos, rango_maximos, 20, n_ingredientes)
print(poblacion_prueba)

[[45.62754891 32.83571182 19.26705241  0.35050024]
 [20.10108496 33.9652345  21.02730082 15.88904933]
 [15.04308651 32.04163718  5.43190919  6.43331184]
 [16.66738039 83.84416361 29.80203762  9.50103763]
 [36.61360294 58.66034756 12.48232989  4.07930574]
 [47.96943165 92.42346506  5.83348508  4.13125047]
 [26.38573205 90.34419148 24.81796837  3.20531991]
 [23.69539437 32.37911292 10.9324318   6.11718506]
 [36.17487578 35.2686222  10.92536974 12.11696291]
 [16.06917337 22.04842594  5.90225973  5.62652291]
 [34.22281133 70.98041525  6.01112329 14.70396063]
 [21.18383683 39.21587579  6.21280575 13.3050711 ]
 [20.76308423 87.14119404 24.55232951  2.26924546]
 [23.0003963  77.71188809 25.88724292  8.04160229]
 [25.50923731 69.71260018 23.46263755 16.86015976]
 [37.92360111 31.77956352 18.06950174 16.89641218]
 [23.11975079 73.44308681  6.29253153 17.56429473]
 [36.43536913 59.82574307  9.06453342 10.44915124]
 [47.00538467 86.2973163  10.90233957  8.66994737]
 [49.76432646 20.04656884 14.77

### 6. Selección de Padres
- Este proceso selecciona dos individuos de la población con probabilidades proporcionales a sus aptitudes. Así, los individuos "mejores" tienen más probabilidades de ser seleccionados como padres.


In [9]:
# Selección de padres
def seleccion(poblacion):
    aptitudes = np.array([calcular_aptitud(ind) for ind in poblacion])
    total_aptitud = np.sum(aptitudes)

    # Evitar división por cero si todas las aptitudes son cero
    if total_aptitud == 0:
        probabilidades = np.ones(len(poblacion)) / len(poblacion)
    else:
        probabilidades = aptitudes / total_aptitud

    # Selecciona dos individuos con probabilidad proporcional a su aptitud
    indices = np.random.choice(len(poblacion), size=2, p=probabilidades)
    return poblacion[indices],probabilidades


In [11]:
seleccion(poblacion=poblacion_prueba)

(array([[36.17487578, 35.2686222 , 10.92536974, 12.11696291],
        [23.0003963 , 77.71188809, 25.88724292,  8.04160229]]),
 array([0.04078514, 0.04573489, 0.18941401, 0.01652678, 0.02839813,
        0.01850357, 0.01587878, 0.18058892, 0.06034627, 0.08391072,
        0.02648951, 0.0869246 , 0.01824447, 0.01791644, 0.01772323,
        0.03045848, 0.02926434, 0.03226869, 0.01504903, 0.04557399]))

### 7. Cruce de Padres
- Esta función implementa el cruce entre dos padres. Si se cumple la probabilidad de cruce, se intercambian segmentos de los padres para generar dos nuevos hijos.

In [8]:
# Cruce de dos padres
def cruce(padre1, padre2):
    if np.random.rand() < cruce_prob:
        punto_cruce = np.random.randint(1, n_ingredientes)
        hijo1 = np.concatenate((padre1[:punto_cruce], padre2[punto_cruce:]))
        hijo2 = np.concatenate((padre2[:punto_cruce], padre1[punto_cruce:]))
        return hijo1, hijo2
    else:
        return padre1, padre2


### 8. Mutación
- La mutación introduce variación en un individuo al modificar uno de sus genes. Luego, se asegura que el valor mutado se mantenga dentro de los rangos permitidos usando `np.clip`.


In [9]:
# Mutación
def mutacion(individuo, rangos_min, rangos_max):
    if np.random.rand() < mutacion_prob:
        punto_mutacion = np.random.randint(n_ingredientes)
        individuo[punto_mutacion] += np.random.standard_normal()
    return np.clip(individuo, rangos_min, rangos_max)


### 9. Algoritmo Genético completo

- La población inicial se somete a un ciclo de generaciones, donde se seleccionan padres, se generan nuevos hijos mediante cruce y mutación, y se realiza un reemplazo parcial con la nueva población.
- Después de cada generación, se selecciona el mejor individuo (la mejor solución) y se imprime su aptitud.


In [49]:
# Algoritmo genético
def algoritmo_genetico():
    poblacion = inicializar_poblacion(rango_minimos, rango_maximos, poblacion_size, n_ingredientes) #Crear una población.
    for generacion in range(generaciones): #Iteración de n generaciones
        nueva_poblacion = [] 
        for _ in range(poblacion_size // 2): #Creación de una nueva población iterando la mitad del tamaño de la población original.
            padre1, padre2 = seleccion(poblacion)
            hijo1, hijo2 = cruce(padre1, padre2)
            nueva_poblacion.append(mutacion(hijo1, rango_minimos, rango_maximos))
            nueva_poblacion.append(mutacion(hijo2, rango_minimos, rango_maximos))
        nueva_poblacion = np.asarray(nueva_poblacion)
        # Reemplazo parcial: seleccionar la mitad mejor de la población actual
        mitad_poblacion_actual = np.argsort([calcular_aptitud(ind) for ind in poblacion])[::-1][:poblacion_size // 2]
        poblacion = np.concatenate((poblacion[mitad_poblacion_actual],
                                   nueva_poblacion)) #Mi nueva población obtenido a partir del reemplazo elitista.
        mejor_individuo = poblacion[np.argmax([calcular_aptitud(ind) for ind in poblacion])]
        mejor_aptitud = calcular_aptitud(mejor_individuo)
        print(f'Generación {generacion + 1}: Mejor aptitud = {mejor_aptitud}, Mejor individuo = {mejor_individuo}')

    return mejor_individuo

# Ejecutar el algoritmo genético
mejor_solucion = algoritmo_genetico()

print('Mejor solución encontrada:')
for i, cantidad in enumerate(mejor_solucion):
    print(f'{ingredientes[i]}: {cantidad:.2f} g')

print('\nInformación nutricional:')
proteina_total, carbohidrato_total, grasa_total, calorias_total = calcular_nutricion(mejor_solucion)
print(f'Proteínas: {proteina_total:.2f} g')
print(f'Carbohidratos: {carbohidrato_total:.2f} g')
print(f'Grasas: {grasa_total:.2f} g')
print(f'Calorías: {calorias_total:.2f} kcal')

costo_total = np.sum(mejor_solucion * precios)
print(f'\nCosto total: ${costo_total:.2f}')


Generación 1: Mejor aptitud = 0.026636546307339304, Mejor individuo = [15.7911099  33.64592733  9.08889038  7.88433092]
Generación 2: Mejor aptitud = 0.03743806304841437, Mejor individuo = [13.82075801 23.45238967 18.05892081  9.57062855]
Generación 3: Mejor aptitud = 0.03743806304841437, Mejor individuo = [13.82075801 23.45238967 18.05892081  9.57062855]
Generación 4: Mejor aptitud = 0.0637014651230416, Mejor individuo = [15.7911099  33.64592733  5.36186907  7.88433092]
Generación 5: Mejor aptitud = 0.0637014651230416, Mejor individuo = [15.7911099  33.64592733  5.36186907  7.88433092]
Generación 6: Mejor aptitud = 0.0637014651230416, Mejor individuo = [15.7911099  33.64592733  5.36186907  7.88433092]
Generación 7: Mejor aptitud = 0.0637014651230416, Mejor individuo = [15.7911099  33.64592733  5.36186907  7.88433092]
Generación 8: Mejor aptitud = 0.0637014651230416, Mejor individuo = [15.7911099  33.64592733  5.36186907  7.88433092]
Generación 9: Mejor aptitud = 0.06655326600929092, M